In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Part 1
### Sally Clark's Story

On the 9th November 1999, Sally Clark (August 1964 – 15 March 2007), then a solicitor and 35-year-old mother of one, was convicted of the murder of her first two children. Christopher, her first child, had died three years earlier, aged just under three months. His death had originally been treated as arising by natural causes, probably “Sudden Infant Death Syndrome” (or “SIDS”). Her second child, Harry, died just over a year later at the age of two months. His death was treated as suspicious by the Home Office pathologist who carried out the post mortem. He then revisited Christopher’s death and determined that that too was suspicious. Sally Clark was arrested some weeks later and eventually tried at Chester Crown Court. The prosecution case relied on flawed statistical evidence presented by paediatrician Professor Sir Roy Meadow, who testified that the chance of two children from an affluent family suffering SIDS was 1 in 73 million. He had arrived at this figure by squaring his estimate of a chance of 1 in 8500 of an individual SIDS death in similar circumstances. The Royal Statistical Society later issued a statement arguing that there was no statistical basis for Meadow's claim, and expressed concern at the "misuse of statistics in the courts".

![Sally Clark](data/sally_clark.png)

In [ ]:
p_death=1/8543

# calculate the probability if two deaths happen independently given p_death
1/(p_death**2)

72982849.00000001

The above evidence is problematic since the two deaths might not be independent at all. Even the cause (or causes) of SIDS is (or are) not fully understood yet, it is possible that there being an unknown, hidden cause which might make a particular family more vulnerable to this disease. In order to explore this case, we need a way to model this situation, here comes the Bayes' Theorem.

### Bayes' Theorem

Bayes' Theorem is a way of finding a probability when we know certain other probabilities. The formula is:

\begin{align*}
P(A\:|\:B) = \frac{P(B\:|\:A) \times P(A)}{P(B)}
\end{align*}

Which tells us: how often A happens given that B happens, written $P(A\:|\:B)$<br>
When we know: how often B happens given that A happens, written $P(B\:|\:A)$<br>
and how likely A is on its own, written $P(A)$<br>
and how likely B is on its own, written $P(B)$<br>

### Reviewing the Sally Clark Case
Now you've learned some basics of the Bayes' Theorem. Let's see how would you defend Sally Clark based on what you have learned. According to Meadow's data,

\begin{align*}
P(death1) = P(death2) \approx \frac{1}{8543}
\end{align*}

There might be underlying factors that are conducive to SIDS deaths within the same family (genetically or environmentally speaking), thus,

\begin{align*}
P(death1\:\&\:death2) \neq P(death1)P(death2) \approx \frac{1}{8543 \times 8543}
\end{align*}

And it is likely that,

\begin{align*}
P(death1\:\&\:death2) > \frac{1}{8543 \times 8543}
\end{align*}

Then,

\begin{align*}
P(death2\:|\:death1) = \frac{P(death1\:\&\:death2)}{P(death1)} > \frac{\frac{1}{8543 \times 8543}}{\frac{1}{8543}} \approx \frac{1}{8543}
\end{align*}

This means that the probability that the second child died from SIDS given the first child died from the same reason is greater than $\frac{1}{8543}$.

Also, Meadow committed a logical fallacy called [confusion of the inverse](https://en.wikipedia.org/wiki/Confusion_of_the_inverse), or the inverse fallacy, who equated a conditional probability with its inverse; that is, given two events A and B, the probability of A happening given that B has happened is assumed to be about the same as the probability of B given A, when there is actually no evidence for this assumption. More formally, P(A|B) is assumed to be approximately equal to P(B|A). According to Meadow, the probability of the two deaths
given that the mother is innocent is miniscule compared with that of the two deaths given that the mother is not.

\begin{align*}
P(death1\:\&\:death2\:|\:innocent) \approx \frac{1}{8543 \times 8543} < P(death1\:\&\:death2\:|\:not\:innocent)
\end{align*}

This could be easily interpreted as a 1 in 73 million chance that Clark was not guilty of murder, which confused $P(death1\:\&\:death2\:|\:innocent)$ with $P(innocent\:|\:death1\:\&\:death2)$, when in actuality,

\begin{align*}
P(innocent\:|\:death1\:\&\:death2) = \frac{P(death1\:\&\:death2\:|\:innocent) \times P(innocent)}{P(death1\:\&\:death2)}
\end{align*}

Notice that $P(innocent)$, the prior probability of the mother’s innocence, should be very close to 1; the vast majority of mothers do not murder their children. Likewise, $P(death1\:\&\:death2)$, the priorprobability of two children in a family dying, should be very close to 0. Therefore,

\begin{align*}
P(innocent\:|\:death1\:\&\:death2) >\!\!> P(death1\:\&\:death2\:|\:innocent)
\end{align*}

## Part 2
### Naive Bayes Classifier

Bayes' Theorem describes how to update the probabilities of hypotheses when given evidence. Let's look at an example and see how it could be applied.

One day, you received an email and how do you know it is spam or not? Or in a more formal term, you want to know the probability of it as a spam, that is, $P(spam)$. Since you haven't read the message yet, so your initial guess would be the chance of number of spam messages in all emails. Let's say it is 10% for demonstration purpose. Thus,

\begin{align*}
P(spam)=0.1
\end{align*}

Correspondingly,

\begin{align*}
P(not\:spam)=1-P(spam)=0.9
\end{align*}

Now you begin to read your new email and see a big word "BUY". Intuitively speaking, $P(spam)$ should increase somehow since this could be a commercial message trying to persuade you to buy something. How could this be reflected using the Bayes' formula? Let's suppose two events:

A: A new email in our inbox is spam<br>
B: The email has a word "BUY"

Plugging them into the Bayes' formula:

\begin{align*}
P(spam\:|\:BUY) = \frac{P(BUY\:|\:spam) \times P(spam)}{P(BUY)}
\end{align*}

We see that the probability of the email as spam given the existence of the word "BUY" could then be updated. We already know $P(spam)$, and we know that the word "BUY" is somewhat frequent in spam emails, that is, $P(BUY\:|\:spam)$ is a reasonably large probability, which seems to make this email more suspicious. But how do we calculate the probability of $P(BUY)$, which is the probability that the word "BUY" happens? On one hand, it is really difficult to do, on the other hand, we simply do not need to do this. If we also plug the event "This new email is not spam" into the Bayes' formula, we get,

\begin{align*}
P(not\:spam\:|\:BUY) = \frac{P(BUY\:|\:not\:spam) \times P(not\:spam)}{P(BUY)}
\end{align*}

We see the two formula above have the same denominator! Thus we do not need to calculate the exact probability of the new email being spam or not but to compare the two numerator to decide which possibility is more likely. We know that the word "BUY" might appear more frequently in spam rather than a non-spam message, that is, $P(BUY\:|\:spam)$ is higher than $P(BUY\:|\:not\:spam)$. Let's suppose $P(BUY\:|\:spam)=0.1$ and $P(BUY\:|\:not\:spam)=0.011$ and plugging them into both numerators.

In [ ]:
# calculate the likelihood of the new email being a spam by multiplying P(BUY | spam) with P(spam)
# and round the result to 3 decimal places
0.1*0.1,0.9*0.011

(0.010000000000000002, 0.009899999999999999)

In [ ]:
# calculate the likelihood of the new email being not a spam by multiplying P(BUY | not spam) with P(not spam)
# and round the result to 3 decimal places


Now we notice that the new email becomes equally likely (or 0.5 in terms of probability) to be spam and not spam by integrating just one tiny piece of evidence (the word "BUY"). Before this evidence came to light, the probability of this email being spam is only 0.1. What if we keep reading the message and integrating more evidence? Let's explore a real-world dataset to find out.

### Spam Filter
#### Exploring the Dataset
Let's start by loading the SMSSpamCollection file:

- Use the read_table() function because the data points are tab separated

- Set header=None because the dataset doesn't have a header row
- Set names=['label', 'text'] to name the columns

In [ ]:
df=pd.read_table("SMSSpamCollection",header=None,names=['label', 'text'])
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Like our previous KNN example, which helps to decide the category of a given iris, we need to separate our data into train and test sets. Generally, the training and test data set is split into an 80:20 ratio. Thus, 20% of the data is set aside for testing purposes.

In [ ]:
# separate data into train and test set, and the ratio of number of cases between train and test is 80:20
x=df['text']
y=df['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)



#### The sklearn way

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

X_train=vectorizer.fit_transform(x_train)
X_test=vectorizer.transform(x_test)

# from sklearn.neighbors import KNeighborsClassifier
# knn=KNeighborsClassifier()
# knn.fit(X_train,y_train)
# np.sum(y_test=="ham")/len(y_test)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes=MultinomialNB()
naive_bayes.fit(X_train,y_train)
np.mean(naive_bayes.predict(X_test)==y_test)

np.float64(0.9874439461883409)

#### Now let's handcraft the algorithm

Write a function get_pw() which takes in a word and return its empirical probabilities in both spam and ham messages, that is, $P(word\:|\:spam)$ and $P(word\:|\:ham)$.

In [ ]:
vectorizer.vocabulary_
X_train

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 59049 stored elements and shape (4457, 7793)>

In [ ]:
def get_pw(w):
  if w not in vectorizer.vocabulary_:
    return 0,0
  ham=X_train[y_train=="ham"]
  spam=X_train[y_train=="spam"]
  hamp=np.sum(ham[:,vectorizer.vocabulary_[w]])/np.sum(ham)
  spamp=np.sum(spam[:,vectorizer.vocabulary_[w]])/np.sum(spam)
  return hamp, spamp

get_pw("abcabc")


(0, 0)

Write another function classify() which takes in a message and use the Bayes' formula to calculate the likelihood ($P(Category) \times P(word_1\:|\:Category) \times P(word_2\:|\:Category)\:...\:\times P(word_n\:|\:Category)$) of the message being both spam and ham. Comparing the results and decide the label of the message.

In [ ]:
def classify(message):
    p_spam=np.mean(y_train=="spam")
    p_ham=np.mean(y_train=="ham")
    for word in message:
      hamp,spamp=get_pw(word)
      if(hamp==0):hamp=0.00001
      if(spamp==0):spamp=0.00001
      p_ham=p_ham*hamp
      p_spam=p_spam*spamp
    if p_ham>p_spam: return "ham"
    else: return "spam"



In [ ]:
# classify all the messages and calculate the prediction accuracy
test_messages = x_test.str.lower().str.replace(',', ' ').str.replace('.', ' ').str.replace('?', ' ').str.replace('!', ' ').str.split()
test_messages

,text
4456,"[storming, msg:, wen, u, lift, d, phne, u, say..."
690,"[<forwarded, from, 448712404000>please, call, ..."
944,"[and, also, i've, sorta, blown, him, off, a, c..."
3768,"[sir, goodmorning, once, free, call, me]"
1189,"[all, will, come, alive, better, correct, any,..."
...,...
2906,"[ha, you, don‘t, know, either, i, did, a, a, c..."
1270,"[tee, hee, off, to, lecture, cheery, bye, bye]"
3944,"[i, got, a, call, from, a, landline, number, i..."
2124,"[+123, congratulations, -, in, this, week's, c..."


In [ ]:
prediction=[]
for message in test_messages:
  prediction.append(classify(message))

In [ ]:
np.mean(prediction==y_test)

np.float64(0.9820627802690582)